In [1]:
import scipy
import numpy
import scipy.io
import matplotlib.pyplot as plt
import pyamg
from scipy.sparse.linalg import cg

In [2]:
A=scipy.io.mmread("/home/yiwei/result_mass_solver/AMGCL_0.08/square_beam_structed_40/bar_sa/P1/ref0/block1/Thread32/0/output/stiffness.mtx")
A_csr=A.tocsr()

In [3]:
b = scipy.io.mmread("/home/yiwei/result_mass_solver/AMGCL_0.08/square_beam_structed_40/bar_sa/P1/ref0/block1/Thread32/0/output/rhs.mtx")

In [4]:
ml = pyamg.smoothed_aggregation_solver(A_csr, max_levels=10, max_coarse=5, keep=True,symmetry='symmetric',strength=[('symmetric', {'theta':0.00})])
print(ml)

multilevel_solver
Number of Levels:     4
Operator Complexity:  1.017
Grid Complexity:      1.018
Coarse Solver:        'pinv2'
  level   unknowns     nonzeros
    0       408483     17571267 [98.31%]
    1         7326       297204 [ 1.66%]
    2          124         4454 [ 0.02%]
    3            2            4 [ 0.00%]



In [5]:
ml_ad = pyamg.aggregation.adaptive_sa_solver(A_csr,pdef=True,num_candidates=6, max_levels=10, max_coarse=500, keep=True,symmetry='symmetric',strength=[('symmetric', {'theta':0.08})])
print(ml_ad)

[multilevel_solver
Number of Levels:     3
Operator Complexity: 11.316
Grid Complexity:      1.618
Coarse Solver:        'pinv2'
  level   unknowns     nonzeros
    0       408483     17571267 [ 8.84%]
    1       252168    181263312 [91.16%]
    2          150         6588 [ 0.00%]
, 796.2812875133022]


In [6]:
def solve_sparse(A, b,M):
    num_iters = 0

    def callback(xk):
        nonlocal num_iters
        num_iters+=1

    x,status=scipy.sparse.linalg.cg(A, b,tol=1e-8, callback=callback,M=M)
    return x,status,num_iters

In [7]:
M = ml.aspreconditioner(cycle='V')  
M_ad = ml.aspreconditioner(cycle='V') # preconditioner
x, info,iter = solve_sparse(A, b, M=M)
x_ad, info_ad,iter_ad = solve_sparse(A, b, M=M_ad)